### Training a model to drive Udacity's self driving car simulator

#### Step 1.: Load the training data

In [5]:
import pandas as pd
import os

data_frame = pd.read_csv(os.path.join('data', 'driving_log.csv'), 
                         names=['cam_centre', 'cam_left', 'cam_right', 'steer_angle', 'throttle', 'brake', 'speed'])
data_frame.head()

,cam_centre,cam_left,cam_right,steer_angle,throttle,brake,speed
0,C:\Users\dev\Documents\IMG\center_2018_03_25_1...,C:\Users\dev\Documents\IMG\left_2018_03_25_14_...,C:\Users\dev\Documents\IMG\right_2018_03_25_14...,0.0,0.0,0.0,0.984607
1,C:\Users\dev\Documents\IMG\center_2018_03_25_1...,C:\Users\dev\Documents\IMG\left_2018_03_25_14_...,C:\Users\dev\Documents\IMG\right_2018_03_25_14...,0.0,0.0,0.0,0.970973
2,C:\Users\dev\Documents\IMG\center_2018_03_25_1...,C:\Users\dev\Documents\IMG\left_2018_03_25_14_...,C:\Users\dev\Documents\IMG\right_2018_03_25_14...,0.0,0.0,0.0,0.957503
3,C:\Users\dev\Documents\IMG\center_2018_03_25_1...,C:\Users\dev\Documents\IMG\left_2018_03_25_14_...,C:\Users\dev\Documents\IMG\right_2018_03_25_14...,0.0,0.0,0.0,0.941993
4,C:\Users\dev\Documents\IMG\center_2018_03_25_1...,C:\Users\dev\Documents\IMG\left_2018_03_25_14_...,C:\Users\dev\Documents\IMG\right_2018_03_25_14...,0.0,0.0,0.0,0.928872


In [6]:
from sklearn.model_selection import train_test_split

X = data_frame[['cam_centre', 'cam_left', 'cam_right']].values
y = data_frame['steer_angle'].values

X_training, X_validation, y_training, y_validation = train_test_split(X, y, test_size=0.2)

### Step 2.: Build a model

In [7]:
from keras.models import Sequential
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten

model = Sequential()
model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(66,200,3)))
model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))
model.summary()

C:\Users\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", strides=(2, 2))`
  
C:\Users\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2))`
  import sys
C:\Users\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2))`
  
C:\Users\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  if __name__ == '__main__':
C:\Users\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  # Remove the CWD from sys.path while we load stuff.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 1, 18, 64)         36928     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 18, 64)         0         
__________

### Step 3.: Training

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from helper import training_generator

checkpointer = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=False,
                                 mode='auto')

model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))
hist = model.fit_generator(training_generator(X_training, y_training, 40, True),
                    samples_per_epoch=20000,
                    nb_epoch=15,
                    validation_data=training_generator(X_validation, y_validation, 40, False),
                    validation_steps=len(X_validation),
                    callbacks=[checkpointer],
                    verbose=1)

C:\Users\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., validation_steps=2571, callbacks=[<keras.ca..., verbose=1, steps_per_epoch=20000, epochs=15)`


Epoch 1/15


C:\Users\dev\Desktop\exp2\helper.py:72: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 160 but corresponding boolean dimension is 66
  mask[(ym - y1) * (x2 - x1) - (y2 - y1) * (xm - x1) > 0] = 1


19999/20000 [============================>.] - ETA: 0s - loss: 0.0424
Epoch 00001: saving model to model-001.h5
20000/20000 [==============================] - 10547s 527ms/step - loss: 0.0424 - val_loss: 0.0690
Epoch 2/15
19999/20000 [============================>.] - ETA: 0s - loss: 0.0395
Epoch 00002: saving model to model-002.h5
20000/20000 [==============================] - 10473s 524ms/step - loss: 0.0395 - val_loss: 0.0695
Epoch 3/15
19999/20000 [============================>.] - ETA: 0s - loss: 0.0389
Epoch 00003: saving model to model-003.h5
20000/20000 [==============================] - 10572s 529ms/step - loss: 0.0389 - val_loss: 0.0691
Epoch 4/15
19999/20000 [============================>.] - ETA: 0s - loss: 0.0384
Epoch 00004: saving model to model-004.h5
20000/20000 [==============================] - 10455s 523ms/step - loss: 0.0384 - val_loss: 0.0689
Epoch 5/15
19999/20000 [============================>.] - ETA: 0s - loss: 0.0382
Epoch 00005: saving model to model-005.h5
